In [23]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import numpy as np
import os
import scipy
from tqdm import trange, tqdm


# add root directory to be able to import packages
# todo: make all packages installable so they can be called/imported by environment
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

from utils.calcium import calcium
from utils.wheel import wheel
from utils.cohort import cohort 

Autosaving every 180 seconds
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
root_dir = '/media/cat/4TB/donato/'
binarization_method = 'upphase'
quiescent_periods = True
cell_randomization = False
recompute = False

#                          #cells, #secs,  #P-day start
animal_ids = {'DON-006084':[200, 600, 19], 
              'DON-006085':[500, 600, 19],
              'DON-003165':[200, 600, 120],
              'DON-003343':[400, 600, 150]
             }


evs = []
times = []
for kk in animal_ids.items():
    
    # 
    animal_id = kk[0]
    
    # 
    n_cells = kk[1][0]
    n_times = kk[1][1]
    p_start = kk[1][2]
    # 
    ad = animal_database.AnimalDatabase()
    ad.load_sessions(animal_id)

    # 
    animal_id = kk[0]
    
    # 
    n_cells = kk[1][0]
    n_times = kk[1][1]
    p_start = kk[1][2]# 
    clrs = ['black','blue','red']

    # 
    binarization_method = 'upphase'
    cum_var = []
    singular_vals = []

    #n_cells = 400
    #n_times = 600
    temp_times = []
    temp_evs = []
    for ctr, session in enumerate(ad.sessions):

        # 
        c = calcium.Calcium()
        c.root_dir = root_dir
        c.data_dir = os.path.join(root_dir, animal_id, session)
        c.animal_id = animal_id
        c.session = session
        c.load_suite2p()

        #c.load_binarization()

        #
        try:
            pca, X_pca = c.load_PCA(session, n_cells, n_times)
        except:
            print ("Pca not available; ", animal_id, session)
            continue
        #
        temp_times.append(ctr+p_start)
        ctr+=1

        #print (session, X_pca.shape)
        cum_var.append(pca.explained_variance_ratio_)

        #
        temp_evs.append(pca.singular_values_)
    
    times.append(temp_times)
    evs.append(temp_evs)

    # 
    print ("# of sessions loaded: ", animal_id, len(cum_var))

In [52]:
##############################################################
##############################################################
##############################################################

#
def get_sessions_from_directory_listing(root_dir,
                                       animal_id):
    
    sessions = np.sort(os.listdir(root_dir+animal_id))
    print ("sessions: ", sessions)
    
    return sessions

def get_slope_log_eigenspectrum(singular_vals):

    # take log
    temp_log = np.log(singular_vals)

    # 
    x = np.arange(len(temp_log))
    z = np.polyfit(x, temp_log, 1)

    #
    slope = z[0]

    return slope
    
    
#
import pickle as pk
from sklearn import metrics

#
animal_ids = [
    'DON-009191',
    'DON-009192',
    'DON-010473',
    'DON-010477'
]

PDays = [
   # [19,28],
   # [19,28],
   # [24,33],
    [16,48],  # expression issues
    [15,60],  
    [16,33],
    [15,33],
]    
    
              

root_dir = '/media/cat/4TB1/donato/'
dir_ = '/002P-F/tif/'

pca_objs = []
pday_times = []
slopes = []
aucs = []

#
for ctra, animal_id in enumerate(animal_ids):

    #
    sessions = get_sessions_from_directory_listing(root_dir,
                                                  animal_id)

    #
    print ("PRocessing: ", animal_id)
    
    #
    pca_objs.append([])
    pday_times.append([])
    slopes.append([])
    aucs.append([])
    
    #
    for ctrs, session in enumerate(sessions):
        #
        #print ("processing: ", session)

        #if session != '20220302':
        #    continue

        c = calcium.Calcium()
        c.root_dir = root_dir
        c.data_dir = os.path.join(root_dir, animal_id, session+dir_, 'suite2p','plane0')
        c.animal_id = animal_id
        c.session = session
        c.detrend_model_order = 1

        #
#             c.load_suite2p()           #note: this already deletes the non-trusted suite2p cells
#             c.save_python = True
#             c.save_matlab = False

#             # 
#             c.load_binarization()


        #
        try:
            #fname_pca = os.path.join(c.data_dir, 'pca_filteredData_goodcellspca.pkl')
            #fname_pca = os.path.join(c.data_dir, 'pca_filteredData_goodcells_2ndhalfpca.pkl')
            #fname_pca = os.path.join(c.data_dir, 'pca_filteredData_goodcells_2ndhalf_top200cellspca.pkl')
            fname_pca = os.path.join(c.data_dir, 'pca_filteredData_goodcells_2ndhalf_top400cellspca.pkl')
            with open(fname_pca, 'rb') as file:
                pca = pk.load(file)
        except:
            print ("Pca not available; ", animal_id, session)
            continue
        
        #
        pca_objs[ctra].append(pca)
        
        # add pday
        pday_times[ctra].append(ctrs+PDays[ctra][0])

        #
        slope = get_slope_log_eigenspectrum(pca.singular_values_)
        slopes[ctra].append(slope)        
        
        #
        var_exp = pca.explained_variance_ratio_
        d = np.arange(var_exp.shape[0])/var_exp.shape[0]
            
        auc = metrics.auc(d,np.cumsum(var_exp))
        aucs[ctra].append(auc)
        #


sessions:  ['20220215' '20220216' '20220217' '20220218' '20220219' '20220220'
 '20220221' '20220222' '20220223' '20220224' '20220225' '20220226'
 '20220227' '20220228' '20220301' '20220302' '20220303' '20220304'
 '20220305' '20220306' '20220307' '20220308' '20220309' '20220310'
 '20220311' '20220312' '20220313' '20220314' '20220315' '20220316'
 '20220317' '20220318']
PRocessing:  DON-009191
sessions:  ['20220215' '20220216' '20220217' '20220218' '20220219' '20220220'
 '20220221' '20220222' '20220223' '20220224' '20220225' '20220226'
 '20220227' '20220228' '20220302' '20220303' '20220304' '20220305'
 '20220306' '20220307' '20220308' '20220309' '20220310' '20220311'
 '20220312' '20220313' '20220314' '20220315' '20220316' '20220317'
 '20220318' '20220319' '20220320' '20220321' '20220322' '20220323'
 '20220324' '20220325' '20220326' '20220327' '20220328' '20220329'
 '20220330' '20220331' '20220401']
PRocessing:  DON-009192
sessions:  ['20220510' '20220511' '20220513' '20220514' '20220515' 

In [53]:
################################################################
######################## PLOT AUCs #############################
################################################################
clrs=['black','blue','red','green']

#
plt.figure(figsize=(8,5))

#
for k in range(len(aucs)):
    x = pday_times[k]
    y = aucs[k]
    plt.plot(x,y,
             c=clrs[k],
            label=animal_ids[k],
            )
    
    #
    z = np.polyfit(x, y, 2)
    p = np.poly1d(z)
    
    #
    times = np.linspace(x[0],x[-1],100)
    #print (times)
    
    #
    plt.plot(times, 
             p(times),
             '--',
             c=clrs[k],
             linewidth=3)

##################################################
##################################################
plt.legend(fontsize=14)
plt.xlabel("PDay", fontsize=14)
plt.xticks(fontsize=14)
plt.ylabel("AUCs", fontsize=14)
plt.suptitle("pca_filteredData_goodcells_2ndhalf_top400cellspca", fontsize=14)
plt.show()

In [ ]:
################################################################
################### PLOT EIGENSPECTRUM #########################
################################################################
plt.figure(figsize=(8,5))

#
for k in range(len(pday_times)):
    x = pday_times[k]
    y = slopes[k]
    
    plt.plot(x,y)

plt.xlabel("PDay", fontsize=14)
plt.xticks(fontsize=14)
plt.ylabel("Slope of log(eigenspectrum)", fontsize=14)
plt.suptitle("All behavior states, all (cleaned) cells", fontsize=14)
plt.show()

In [36]:
F = np.load('/media/cat/4TB1/donato/DON-009191/20220223/002P-F/tif/suite2p/plane0/F.npy')
print (F.shape)

idx = np.load('/media/cat/4TB1/donato/DON-009191/20220223/002P-F/tif/suite2p/plane0/iscell.npy')
print (np.where(idx==1.0)[0].shape)

(632, 111496)
(427,)
